# Zadanie 4 - Odwzorowanie Gaussa-Krügera: układy współrzędnych płaskich stosowanych w Polsce

<li>Autor: Adrian Fabisiewicz</li>
<li>Numer indeksu: 328935</li>

## Cel ćwiczenia
Celem ćwiczenia jest obliczenie współrzędnych płaskich w układach PL-1992 oraz PL-2000 czterech punktów, których współrzędne geodezyjne $\phi$, $\lambda$ zostały wcześniej wyznaczone na podstawie zadania wprost przeniesienia współrzędnych na elipsoidzie. Należało również wyznaczyć redukcję długości, obliczonych na powierzchni układu PL-2000 oraz PL-1992, na powierzchnię elipsoidy. Na koniec, stosując wzory Gaussa, należało wyznaczyć pole powierzchni czworoboku na płaszczyźnie układów PL-1992 oraz PL-2000. Z wykorzystaniem pythonowej biblioteki <i>pyproj</i> należało przeliczyć współrzędne punktów do układu równopolowego PL-LAEA oraz policzyć pole powierzchni w tym układzie. Mając te dane można było wykonać odpowiednie analizy i porównania i wyciągnąć odpowiednie wnioski.

## Dane do zadania
Dany były współrzędne geodezyjne φ, λ czterech punktów, które zostały wyznaczone w zadaniu wprost przeniesienia współrzędnych na elipsoidzie metodą Kivioji.


<table border="1">
    <thead>
        <tr>
            <th>numer punktu</th>
            <th>φ</th>
            <th>λ</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>1</td>
            <td>51°00′00.00000”</td>
            <td>19°00′00.00000”</td>
        </tr>
        <tr>
            <td>2</td>
            <td>51°21'34.36205"</td>
            <td>19°00'00.00000"</td>
        </tr>
        <tr>
            <td>3</td>
            <td>51°21'2.70063"</td>
            <td>20°26'7.76249"</td>
        </tr>
        <tr>
            <td>4</td>
            <td>50°59'28.33662"</td>
            <td>20°26'7.76249"</td>
        </tr>
    </tbody>
</table>

## Kolejność wykonywania obliczeń

### Zapisanie danych używanych w zadaniu
Aby nie zapisywać wielokrotnie tych samych stałych zmiennych, zapisałem je na początku kodu. Są to między innymi takie dane jak: dłuższa i krótsza półoś elipsoidy czy pierwszy i drugi mimośród.

In [83]:
import math
import numpy as np

e2 = 0.00669438002290
a = 6378137
b2 = a**2 * (1 - e2)
eprim2 = (a**2 - b2) / b2

A0 = 1 - e2/4 - 3*e2**2/64 - 5*e2**3/256
A2 = 3/8 * (e2 + e2**2/4 + 15*e2**3/128)
A4 = 15/256 * (e2**2 + 3*e2**3/4)
A6 = 35*e2**3/3072

### Stworzenie funkcji na podstawie algorytmu przeliczenia współrzędnych φ i λ na płaszczyznę odwzorowania Gaussa-Krügera - przeliczenie wprost (x<sub>gk</sub>, y<sub>gk</sub>) = f(φ, λ)
Funkcja jako argumenty przyjmuje współrzędne geodezyjne φ i λ oraz λ<sub>0</sub>, czyli odpowiedni południk osiowy układu. Zwraca współrzędne płaskie x<sub>gk</sub> i y<sub>gk</sub> w układzie Gaussa-Krügera.

In [84]:
def to_gk(fi, lam, lam0):
    fi = np.deg2rad(fi)
    lam = np.deg2rad(lam)
    e2 = 0.0066943800290
    a = 6378137

    b2 = a ** 2 * (1 - e2)
    e2_prim = (a ** 2 - b2) / b2
    lam0 = np.deg2rad(lam0)
    t = np.tan(fi)
    N = a / np.sqrt(1 - e2 * np.sin(fi) ** 2)
    eta2 = e2_prim * (np.cos(fi) ** 2)
    delta_lam = lam - lam0

    A0 = 1 - e2 / 4 - 3 * e2 ** 2 / 64 - 5 * e2 ** 3 / 256
    A2 = 3 / 8 * (e2 + e2 ** 2 / 4 + 15 * e2 ** 3 / 128)
    A4 = 15 / 256 * (e2 ** 2 + 3 * e2 ** 3 / 4)
    A6 = 35 * e2 ** 3 / 3072

    sigma = a * (A0 * fi - A2 * np.sin(2 * fi) + A4 * np.sin(4 * fi) - A6 * np.sin(6 * fi))

    x = sigma + delta_lam ** 2 / 2 * N * np.sin(fi) * np.cos(fi) * (1 + delta_lam ** 2 / 12 * (np.cos(fi) ** 2) * (5 -
        t ** 2 + 9 * eta2 + 4 * eta2 ** 2) + delta_lam ** 4 / 360 * (np.cos(fi) ** 4) * (61 - 58 * t ** 2 + t ** 4 +
        270 * eta2 - 330 * eta2 * t ** 2))

    y = delta_lam * N * np.cos(fi) * (
                1 + delta_lam ** 2 / 6 * np.cos(fi) ** 2 * (1 - t ** 2 + eta2) + delta_lam ** 4 / 120
                * np.cos(fi) ** 4 * (5 - 18 * t ** 2 + t ** 4 + 14 * eta2 - 58 * eta2 * t ** 2))
    return x, y

### Funkcja wykonująca przeliczenie odwrotne: (φ, λ) = f (x<sub>gk</sub>, y<sub>gk</sub>)
Funkcja jako argumenty przyjmuje współrzędne płaskie x<sub>gk</sub> i y<sub>gk</sub> oraz λ<sub>0</sub>, czyli odpowiedni południk osiowy układu. Zwraca współrzędne geodezyjne φ i λ.

In [85]:
def from_gk(xgk, ygk, lam0):
    fi = xgk / (a * A0)
    sigma = a * (A0 * fi - A2 * np.sin(2 * fi) + A4 * np.sin(4 * fi) - A6 * np.sin(6 * fi))

    while True:
        fi1 = fi + (xgk - sigma) / (a * A0)

        N = a / math.sqrt(1-e2*np.sin(fi1)**2)
        M = a * (1 - e2) / math.sqrt(1-e2*np.sin(fi1)**2)**3
        t = math.tan(fi1)
        eta2 = eprim2 * np.cos(fi1)**2
        sigma = a * (A0 * fi1 - A2 * np.sin(2 * fi1) + A4 * np.sin(4 * fi1) - A6 * np.sin(6 * fi1))

        if abs(fi1 - fi) < (0.000001 / 3600):
            break

        fi = fi1

    fi = fi1 - ((ygk**2 * t) / (2 * M * N)) * (1 - ygk**2 / 12 * N**2 * (5 + 3*t**2 + eta2 - 9*t**2*eta2 - 4*eta2**2) + ygk**4 / 360 * N**4 * (61 + 90*t**2 + 45*t**4))

    lam = lam0 + (ygk / (N * np.cos(fi))) * (1 - ygk**2 / 6 * N**2 * (1 + 2*t**2 + eta2) + ygk**4 / 120 * N**4 * (5 + 28*t**2 + 24*t**4 + 6*eta2 + 8*t**2*eta2))

    return fi, lam

### Funkcja, obliczająca współrzędne płaskie punktu w układzie PL-2000
Funkcja jako argumenty przyjmuje współrzędne geodezyjne φ i λ oraz λ<sub>0</sub>, czyli odpowiedni południk osiowy układu. W środku wywołuje funkcję przeliczającą współrzędne geodezyjne na płaskie w układzie Gaussa-Krügera. Sprawdza, w którym z 4 pasów południkowych układu PL-2000 znajduje się punkt, a następnie oblicza współrzędne płaskie w układzie PL-2000. Zwraca współrzędne płaskie x<sub>2000</sub> i y<sub>2000</sub> w układzie PL-2000.

In [86]:
def to_2000(fi, lam, lam0):
    xgk, ygk = to_gk(fi, lam, lam0)
    m0 = 0.999923

    nr = 6

    x2000 = m0 * xgk
    y2000 = m0 * ygk + nr * 1000000 + 500000

    return x2000, y2000

### Funkcja, obliczająca współrzędne płaskie punktu w układzie PL-1992
Funkcja jako argumenty przyjmuje współrzędne geodezyjne φ i λ oraz λ<sub>0</sub>, czyli odpowiedni południk osiowy układu. W środku wywołuje funkcję przeliczającą współrzędne geodezyjne na płaskie w układzie Gaussa-Krügera. Następnie oblicza współrzędne płaskie w układzie PL-1992. Zwraca współrzędne płaskie x<sub>1992</sub> i y<sub>1992</sub> w układzie PL-1992.

In [87]:
def to_1992(fi, lam, lam0):
    xgk, ygk = to_gk(fi, lam, lam0)
    m0 = 0.9993

    x1992 = m0 * xgk - 5300000
    y1992 = m0 * ygk + 500000

    return x1992, y1992

### Obliczenie współrzędnych punktów w układach PL-1992, PL-2000 oraz Gaussa-Krügera

In [88]:
points = [[51.0, 19.0], [51.35954501388889, 19.0], [51.350750175, 20.435489580555554], [50.991204616666664, 20.435489580555554]]

gk = []
pl2000 = []
pl1992 = []

import pandas as pd

for point in points:
    fi = point[0]
    lam = point[1]

    print("Punkt numer", points.index(point)+1)

    xgk, ygk = to_gk(fi, lam, 19)
    gk.append([xgk, ygk])
    print("GK: ", '{0:.3f}'.format(xgk), '{0:.3f}'.format(ygk))

    x2000, y2000 = to_2000(fi, lam, 18)
    pl2000.append([x2000, y2000])
    print("2000: ", '{0:.3f}'.format(x2000), '{0:.3f}'.format(y2000))

    x1992, y1992 = to_1992(fi, lam, 19)
    pl1992.append([x1992, y1992])
    print("1992: ", '{0:.3f}'.format(x1992), '{0:.3f}'.format(y1992))
    print("")

Punkt numer 1
GK:  5652085.723 0.000
2000:  5652126.564 6570191.535
1992:  348129.263 500000.000

Punkt numer 2
GK:  5692085.723 0.000
2000:  5692122.186 6569647.614
1992:  388101.263 500000.000

Punkt numer 3
GK:  5692085.723 100004.092
2000:  5693485.762 6669649.212
1992:  388101.263 599934.089

Punkt numer 4
GK:  5652088.396 100784.906
2000:  5653496.546 6670974.216
1992:  348131.934 600714.357



### Obliczenie długości odcinków między punktami na płaszczyznach układów PL-1992, PL-2000 oraz Gaussa-Krügera

In [89]:
def distance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

distances_gk = []
distances_pl2000 = []
distances_pl1992 = []

for i in range(len(gk)):
    if i == 3:
        distances_gk.append(distance(gk[i], gk[0]))
        distances_pl2000.append(distance(pl2000[i], pl2000[0]))
        distances_pl1992.append(distance(pl1992[i], pl1992[0]))
        break
    distances_gk.append(distance(gk[i], gk[i+1]))
    distances_pl2000.append(distance(pl2000[i], pl2000[i+1]))
    distances_pl1992.append(distance(pl1992[i], pl1992[i+1]))

print("Długości odcinków między punktami na płaszczyźnie PL2000:")
print(distances_pl2000)
print('')

print("Długości odcinków między punktami na płaszczyźnie PL1992:")
print(distances_pl1992)
print('')

print("Długości odcinków między punktami na płaszczyźnie GK:")
print(distances_gk)
print('')

Długości odcinków między punktami na płaszczyźnie PL2000:
[39999.32017877123, 100010.89433113993, 40011.16206806733, 100791.99213772782]

Długości odcinków między punktami na płaszczyźnie PL1992:
[39971.99990267865, 99934.08906411426, 39976.9448238994, 100714.35675931034]

Długości odcinków między punktami na płaszczyźnie GK:
[39999.99990261067, 100004.0919284642, 40004.948287701045, 100784.90619364585]



### Obliczenie redukcji długości

In [90]:
m0_1992 = 0.9993

length_gk = []

for distance in distances_pl1992:
    length_gk.append(distance / m0_1992)

print("Długości odcinków na płaszczyźnie GK w 1992:")
print(length_gk)
print('')

middle_phis = []
middle_ms = []
middle_ns = []

for i in range(0, len(points)):
    if i == len(points)-1:
        middle_phis.append((points[i][0] + points[0][0]) / 2)
    else:
        middle_phis.append((points[i][0] + points[i+1][0]) / 2)

middle_phis = np.deg2rad(middle_phis)

for phi in middle_phis:
    middle_ms.append(a * (1 - e2) / np.sqrt(1-e2*np.sin(phi)**2)**3)
    middle_ns.append(a / np.sqrt(1-e2*np.sin(phi)**2))

reductions_1992 = []
for i in range(0, len(points)):
    if i == len(points)-1:
        rAB = distances_pl1992[i] * (gk[i][1]**2 + gk[i][1]*gk[0][1] + gk[0][1]**2) / (6 * middle_ms[i] * middle_ns[i])
    else:
        rAB = distances_pl1992[i] * (gk[i][1]**2 + gk[i][1]*gk[i+1][1] + gk[i+1][1]**2) / (6 * middle_ms[i] * middle_ns[i])
    reductions_1992.append(rAB)

print("Redukcje długości odcinków na płaszczyźnie GK w 1992:")
print(reductions_1992)

Długości odcinków na płaszczyźnie GK w 1992:
[39999.999902610485, 100004.09192846419, 40004.94828770079, 100784.90619364589]

Redukcje długości odcinków na płaszczyźnie GK w 1992:
[0.0, 4.088585643486725, 4.945327968503748, 4.18544909275307]


In [131]:

#2000
m0_2000 = 0.999923

length_gk = []

for distance in distances_pl2000:
    length_gk.append(distance / m0_2000)

print("Długości odcinków między punktami na płaszczyźnie GK w 2000:")
print(length_gk)
print('')

middle_phis = []
middle_ms = []

middle_phis = []
middle_ms = []
middle_ns = []

for i in range(0, len(points)):
    if i == len(points)-1:
        middle_phis.append((points[i][0] + points[0][0]) / 2)
    else:
        middle_phis.append((points[i][0] + points[i+1][0]) / 2)

middle_phis = np.deg2rad(middle_phis)

for phi in middle_phis:
    middle_ms.append(a * (1 - e2) / np.sqrt(1-e2*np.sin(phi)**2)**3)
    middle_ns.append(a / np.sqrt(1-e2*np.sin(phi)**2))

reductions_2000 = []
for i in range(0, len(points)):
    if i == len(points)-1:
        rAB = distances_pl2000[i] * (gk[i][1]**2 + gk[i][1]*gk[0][1] + gk[0][1]**2) / (6 * middle_ms[i] * middle_ns[i])
    else:
        rAB = distances_pl2000[i] * (gk[i][1]**2 + gk[i][1]*gk[i+1][1] + gk[i+1][1]**2) / (6 * middle_ms[i] * middle_ns[i])
    reductions_2000.append(rAB)

print("Redukcje długości odcinków na płaszczyźnie GK w 2000:")
print(reductions_2000)

Długości odcinków między punktami na płaszczyźnie GK w 2000:
[40002.40036359923, 100018.59576301368, 40014.24316479102, 100799.75371876417]

Redukcje długości odcinków na płaszczyźnie GK w 2000:
[0.0, 4.091727963740466, 4.949560795582811, 4.188675434405031]


### Obliczenie długości odcinków między punktami na powierzchni elipsoidy

In [132]:
length_1992 = []
for i in range(0, len(points)):
    length_1992.append(length_gk[i] - reductions_1992[i])

print("Długości odcinków na elipsoidzie w PL-1992:")
print(length_1992)

Długości odcinków na elipsoidzie w PL-1992:
[40002.40036359923, 100014.5071773702, 40009.297836822516, 100795.56826967142]


In [133]:
length_2000 = []
for i in range(0, len(points)):
    length_2000.append(length_gk[i] - reductions_2000[i])

print("Długości odcinków na elipsoidzie w PL-2000:")
print(length_2000)

Długości odcinków na elipsoidzie w PL-2000:
[40002.40036359923, 100014.50403504995, 40009.29360399544, 100795.56504332977]


### Obliczenie pól powierzchni czworoboku na płaszczyźnie układów PL-1992 oraz PL-2000

In [134]:
polepl2000 = (pl2000[1][0] * (pl2000[2][1] - pl2000[0][1]) + pl2000[2][0] * (pl2000[3][1] - pl2000[1][1]) + pl2000[3][0] * (pl2000[0][1] - pl2000[2][1]) + pl2000[0][0] * (pl2000[1][1] - pl2000[3][1])) / 2
print("Pole powierzchni na płaszczyźnie PL-2000: ", '{0:.2f}'.format(polepl2000), "m^2, czyli ", '{0:.2f}'.format(polepl2000/1000000), "km^2")

polepl1992 = (pl1992[1][0] * (pl1992[2][1] - pl1992[0][1]) + pl1992[2][0] * (pl1992[3][1] - pl1992[1][1]) + pl1992[3][0] * (pl1992[0][1] - pl1992[2][1]) + pl1992[0][0] * (pl1992[1][1] - pl1992[3][1])) / 2
print("Pole powierzchni na płaszczyźnie PL-1992: ", '{0:.2f}'.format(polepl1992), "m^2, czyli ", '{0:.2f}'.format(polepl1992/1000000), "km^2")

Pole powierzchni na płaszczyźnie PL-2000:  4016201746.26 m^2, czyli  4016.20 km^2
Pole powierzchni na płaszczyźnie PL-1992:  4010026395.26 m^2, czyli  4010.03 km^2


### Transformacja współrzędnych punktów do układu PL-LAEA z wykorzystaniem biblioteki pyproj oraz obliczenie pola powierzchni czworoboku na płaszczyźnie układu PL-LAEA

In [135]:
from pyproj import CRS, Transformer

output_proj = CRS.from_epsg(3035)
input_proj = CRS.from_epsg(4326)

transformer = Transformer.from_crs(input_proj, output_proj)

plalea = []
for point in points:
    plalea.append(transformer.transform(point[0], point[1]))

polepllaea = (plalea[1][0] * (plalea[2][1] - plalea[0][1]) + plalea[2][0] * (plalea[3][1] - plalea[1][1]) + plalea[3][0] * (plalea[0][1] - plalea[2][1]) + plalea[0][0] * (plalea[1][1] - plalea[3][1])) / 2

print("Pole powierzchni na płaszczyźnie PL-LAEA: ", '{0:.2f}'.format(polepllaea), "m^2, czyli ", '{0:.2f}'.format(polepllaea/1000000), "km^2")

Pole powierzchni na płaszczyźnie PL-LAEA:  4015252907.09 m^2, czyli  4015.25 km^2


### Zestawienie wyników

#### Tabela przedstawiająca współrzędne geodezyjne punktów oraz obliczone współrzędne układach PL-1992, PL-2000 oraz Gaussa-Krügera

In [136]:
wf = pd.DataFrame([1, 2, 3, 4], columns=['Punkt'])
wf['φ'] = pd.Series([x[0] for x in points])
wf['λ'] = pd.Series([x[1] for x in points])
wf['Xgk'] = pd.Series([x[0] for x in gk])
wf['Ygk'] = pd.Series([x[1] for x in gk])
wf['X2000'] = pd.Series([x[0] for x in pl2000])
wf['Y2000'] = pd.Series([x[1] for x in pl2000])
wf['X1992'] = pd.Series([x[0] for x in pl1992])
wf['Y1992'] = pd.Series([x[1] for x in pl1992])
wf.index = wf.index + 1 
pd.set_option("display.float_format", lambda x: "%.3f" % x)
wf

,Punkt,φ,λ,Xgk,Ygk,X2000,Y2000,X1992,Y1992
1,1,51.000,19.000,5652085.723,0.000,5652126.564,6570191.535,348129.263,500000.000
2,2,51.360,19.000,5692085.723,0.000,5692122.186,6569647.614,388101.263,500000.000
3,3,51.351,20.435,5692085.723,100004.092,5693485.762,6669649.212,388101.263,599934.089
4,4,50.991,20.435,5652088.396,100784.906,5653496.546,6670974.216,348131.934,600714.357


#### Tabela przedstawiająca długość odcinków na elipsoidzie po wykonaniu redukcji długości

In [137]:
c = [40000.000, 100000.000, 40000.000, 100000.000]
dene = pd.DataFrame(['1-2', '2-3', '3-4', '4-1'], columns=['Odcinek'])
dene['Długość linii geodezyjnej'] = pd.Series(x for x in c)
dene['Długość odcinka na elipsoidzie w 1992'] = pd.Series(length_1992)
dene['Długość odcinka na elipsoidzie w 2000'] = pd.Series(length_2000)
dene.index = dene.index + 1
pd.set_option("display.float_format", lambda x: "%.5f" % x)
dene

,Odcinek,Długość linii geodezyjnej,Długość odcinka na elipsoidzie w 1992,Długość odcinka na elipsoidzie w 2000
1,1-2,40000.00000,40002.40036,40002.40036
2,2-3,100000.00000,100014.50718,100014.50404
3,3-4,40000.00000,40009.29784,40009.29360
4,4-1,100000.00000,100795.56827,100795.56504


Mniejsze zniekształcenie dla odcinków 2-3, 3-4, 4-1 występuje w układzie PL-2000, natomiast dla odcinka 1-2 zniekształcenie jest takie same dla obu układów.

#### Zestawienie pól powierzchni czworoboku w układach PL-2000, PL-1992, PL-LAEA oraz na powierzchni elipsoidy

In [138]:
pola = pd.DataFrame([polepl2000, polepl1992, polepllaea], columns=['Pole [km²]'])
pola['Układ'] = ['PL-2000', 'PL-1992', 'PL-LAEA']
pola['Pole [km²]'] = pola['Pole [km²]'] / 1000000
#dodaj wiersz
pola.loc[4] = [4113.295698406223, 'Elipsoida']
pd.set_option("display.float_format", lambda x: "%.3f" % x)
pola

,Pole [km²],Układ
0,4016.202,PL-2000
1,4010.026,PL-1992
2,4015.253,PL-LAEA
4,4113.296,Elipsoida
